In [145]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [146]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time 
from bs4 import BeautifulSoup


driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://19hz.info/eventlisting_BayArea.php#crewList')
driver.find_element_by_xpath('/html/body/div[1]/div/span[1]/button[1]').click()
time.sleep(5)
body = driver.find_element_by_xpath('/html/body')

events = []
recurring_events = []
table = driver.find_element(By.XPATH, '/html/body/table[1]/tbody')
rows = table.find_elements(By.TAG_NAME, "tr") # get all of the rows in the table
for row in rows:
    events.append(row.text)

rec_table = driver.find_element_by_id("mobileEnable")
rows = rec_table.find_elements(By.TAG_NAME, "tr") # get all of the rows in the table
for row in rows:
    recurring_events.append(row.text)


In [147]:
text = body.text.split('\n')
start_index,end_index = 0,0
for i in range(len(text)):
    if text[i] == 'Venue List':
        start_index = i+2
    if text[i] == 'Promoter List':
        end_index = i
        break
text_comp = text[start_index:end_index]
venues = {}
for line in text_comp:
    name,address = line.split('-')
    name = name.rstrip()
    address = address.lstrip()
    venues[name] = address
driver.quit()

In [148]:
from opencage.geocoder import OpenCageGeocode
from shapely.geometry import Point


key = '21b734acde524209895d1c01fc8e6eea'
geocoder = OpenCageGeocode(key)
locations = {}

for k,v in venues.items():
    results = geocoder.geocode(v)
    loc = Point(results[0]['geometry']['lat'],results[0]['geometry']['lng'])
    locations[k] = ((v,loc))


In [150]:
def generate_prompt(event):
    prompt = """Extract the Date/Time, Event Name, Venue, Music Genres, Price, Age Requirement, and Performers from each event and store it into a JSON with the following structure {venue: venue_name, event: event_name, date: date/time, age: age_requirement, price: price, music:[musc genres], performers:[performers]}""" + '\n' + "Event:" + event + '\n' + 'Enclose each key and value in the JSON structure with double quotes.'
    return prompt

def find_address(venue,locations):
    prompt = "Find the closest match to the venue given the list of locations. Select that location from the list of locations" + '\n' + 'Venue:' + venue + '\n' + 'Locations:' + str(locations)
    return prompt

In [160]:
import json
import pandas as pd
import numpy as np

sf_events = pd.DataFrame()


for event in events:
   
   response = openai.Completion.create(
        model="text-davinci-003",
        prompt=generate_prompt(event),
        temperature=0.0,
        max_tokens = 200,
   )
   response_text = response['choices'][0]['text'].strip()
   try:
     response_dict = json.loads(response_text)
   except:
     print(response_text)    
     break

   venue = response_dict['venue']

   response = openai.Completion.create(
        model="text-curie-001",
        prompt=find_address(venue,locations.keys()),
        temperature=0.0,
        max_tokens = 100,
   )

   location = response['choices'][0]['text'].strip()
   try:
     response_dict['location'] = locations[location]
   except:
     response_dict['location'] = np.nan

 
   sf_events = sf_events.append(response_dict, ignore_index=True)
   


     


{
  "venue": "The Midway",
  "event": "P.O.S - Deeper Tales Tour",
  "date": "Sun: Mar 19 (2pm-8pm)",
  "age": 21+,
  "price": "$40",
  "music": [
    "progressive house",
    "trance"
  ],
  "performers": []
}


In [ ]:
sf_events.to_csv('bar_events.csv')